In [1]:
using ReinforcementLearning
using Flux
using Statistics
using Plots

┌ Info: CUDAdrv.jl failed to initialize, GPU functionality unavailable (set JULIA_CUDA_SILENT or JULIA_CUDA_VERBOSE to silence or expand this message)
└ @ CUDAdrv C:\Users\mclau\.julia\packages\CUDAdrv\b1mvw\src\CUDAdrv.jl:67


In [2]:
env = CartPoleEnv(;T=Float32, seed=123)

CartPoleEnv{Float32}(gravity=9.8,masscart=1.0,masspole=0.1,totalmass=1.1,halflength=0.5,polemasslength=0.05,forcemag=10.0,tau=0.02,thetathreshold=0.20943952,xthreshold=2.4,max_steps=200)

In [3]:
model = Chain(Dense(4, 28, relu),
              Dense(28, 28, relu),
              Dense(28, 2))

Chain(Dense(4, 28, relu), Dense(28, 28, relu), Dense(28, 2))

In [15]:
test=DQN(0.95,0.01,32,150,4,2,1.0,20,model)

DQN(true, true, 1.0, 0.95, 0.01, 32, Any[], 150, 4, 2, 1.0, 20, Chain(Dense(4, 28, relu), Dense(28, 28, relu), Dense(28, 2)), Chain(Dense(4, 28, relu), Dense(28, 28, relu), Dense(28, 2)), var"#loss#3"{Chain{Tuple{Dense{typeof(relu),Array{Float32,2},Array{Float32,1}},Dense{typeof(relu),Array{Float32,2},Array{Float32,1}},Dense{typeof(identity),Array{Float32,2},Array{Float32,1}}}}}(Chain(Dense(4, 28, relu), Dense(28, 28, relu), Dense(28, 2))), Params([Float32[-0.036902934 0.4196585 -0.19523771 -0.32187587; -0.10929825 -0.06586251 -0.25376916 -0.17929928; … ; 0.39383727 -0.35221875 -0.16807346 0.042578354; -0.08568105 0.27010125 -0.23473038 -0.4302871], Float32[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], Float32[-0.002151975 0.20309353 … -0.007061678 -0.21619923; 0.046937093 0.17890884 … 0.013441279 -0.24230918; … ; 0.2758331 -0.13845967 … -0.012044739 0.110552676; 0.2953952 0.16687441 … 0.2599262 -0.007538975], Float32[0.0, 0.0, 

In [20]:
main_DQN(test,env)

Episode: 1 | Score: 13.0 
Episode: 2 | Score: 9.0 
Episode: 3 | Score: 9.0 
Episode: 4 | Score: 13.0 
Episode: 5 | Score: 8.0 
Episode: 6 | Score: 9.0 
Episode: 7 | Score: 8.0 
Episode: 8 | Score: 8.0 
Episode: 9 | Score: 8.0 
Episode: 10 | Score: 11.0 
Episode: 11 | Score: 8.0 
Episode: 12 | Score: 13.0 
Episode: 13 | Score: 9.0 
Episode: 14 | Score: 10.0 
Episode: 15 | Score: 12.0 
Episode: 16 | Score: 8.0 
Episode: 17 | Score: 8.0 
Episode: 18 | Score: 8.0 
Episode: 19 | Score: 9.0 
Episode: 20 | Score: 7.0 
Episode: 21 | Score: 8.0 
Episode: 22 | Score: 9.0 
Episode: 23 | Score: 9.0 
Episode: 24 | Score: 9.0 
Episode: 25 | Score: 7.0 
Episode: 26 | Score: 8.0 
Episode: 27 | Score: 8.0 
Episode: 28 | Score: 9.0 
Episode: 29 | Score: 9.0 
Episode: 30 | Score: 9.0 
Episode: 31 | Score: 9.0 
Episode: 32 | Score: 8.0 
Episode: 33 | Score: 9.0 
Episode: 34 | Score: 8.0 
Episode: 35 | Score: 9.0 
Episode: 36 | Score: 7.0 
Episode: 37 | Score: 8.0 
Episode: 38 | Score: 9.0 
Episode: 39 | S

Episode: 201 | Score: 9.0 Last 100 episodes mean score: 8.45
Problem unsolved!


(201, Any[13.0f0, 9.0f0, 9.0f0, 13.0f0, 8.0f0, 9.0f0, 8.0f0, 8.0f0, 8.0f0, 11.0f0  …  9.0f0, 7.0f0, 9.0f0, 9.0f0, 7.0f0, 8.0f0, 9.0f0, 9.0f0, 8.0f0, 9.0f0])

In [14]:
mutable struct DQN
    TRAIN::Bool
    CHANGE::Bool
    ϵ::Float64
    ϵ_DECAY::Float64
    ϵ_MIN::Float64
    BATCH_SIZE::Int64
    MEMORY
    MEM_SIZE::Int64
    STATE_SIZE::Int64
    ACTION_SIZE::Int64
    γ::Float64
    C_UPDATE::Int64
    model1
    model2
    loss
    ps
    opt
    
    function DQN(ϵ_DECAY::Float64, ϵ_MIN::Float64,BATCH_SIZE::Int64,MEM_SIZE::Int64,STATE_SIZE::Int64,
            
                 ACTION_SIZE::Int64,γ::Float64,C_UPDATE::Int64, model1)
        loss(x,y)=Flux.mse(model1(x), y)
        
        new(true, true, 1.0f0, ϵ_DECAY, ϵ_MIN, BATCH_SIZE, [], MEM_SIZE, STATE_SIZE, ACTION_SIZE,γ,C_UPDATE,
            
            model1, deepcopy(model1),loss,Flux.params(model1), ADAM(0.001, (0.9, 0.8)))
    end
    
end


In [21]:

#Fonctions élémentaires

#OK
function action(dqn::DQN,state::Array{Float32,1})

  if rand() <= dqn.ϵ && dqn.TRAIN
        
    return rand(1:dqn.ACTION_SIZE)

  end



  act_values = dqn.model1(state)

  return Flux.argmax(act_values)
    
end

#OK
function update_ϵ!(dqn::DQN)
    
    x = dqn.ϵ*dqn.ϵ_DECAY
    
    if x < dqn.ϵ_MIN && dqn.CHANGE
        
        dqn.ϵ=dqn.ϵ_MIN
        
        dqn.CHANGE=false
        
        
    elseif dqn.CHANGE
        
        dqn.ϵ = x
        
    end
    
    
end

#OK
function act(action::Int64, env)
    
    env(action)
    
    
    obs=observe(env)
    
    get_state(obs), get_reward(obs), get_terminal(obs)
    
end

    
#OK    
function remember!(dqn::DQN,state::Array{Float32,1}, action::Int64, reward::Float32, next_state::Array{Float32,1}, done::Bool)

  if length(dqn.MEMORY) == dqn.MEM_SIZE

    deleteat!(dqn.MEMORY, 1)

  end

  push!(dqn.MEMORY, (state, action, reward, next_state, done))

end


function replay!(dqn::DQN)

  batch_size = min(dqn.BATCH_SIZE, length(dqn.MEMORY))

  minibatch = sample(dqn.MEMORY, batch_size, replace = false)

  

  x = Matrix{Float32}(undef,dqn.STATE_SIZE, batch_size)

  y = Matrix{Float32}(undef,dqn.ACTION_SIZE, batch_size)

  for (iter, (state, action, reward, next_state, done)) in enumerate(minibatch)

    target = reward

    if !done

      target += dqn.γ * maximum(dqn.model2(next_state))

    end



    target_f = dqn.model1(state)

    target_f[action] = target

    

    x[:, iter] .= state

    y[:, iter] .= target_f

  end

    Flux.train!(dqn.loss,dqn.ps, [(x, y)], dqn.opt)
    

end


function copy(iter::Int64, dqn::DQN)
    
    if iter%dqn.C_UPDATE==0
        
        dqn.model2 = deepcopy(dqn.model1)
        
    end
    
end
    
    
#Run 1 épisode

function episode!(dqn::DQN, env)
    
    obs_ini=observe(env)
    
    current_state=get_state(obs_ini)
    
    
    total_reward=0
    
    i = 0
    
    while true
        
        current_action=action(dqn,current_state)
        
        current_next_state,current_reward,current_done=act(current_action,env)
        
        
        total_reward+=current_reward
        
        
        remember!(dqn, current_state, current_action, current_reward, current_next_state, current_done)
        
        current_state=current_next_state
        
        replay!(dqn)
        
        update_ϵ!(dqn)
        
        i+=1
        
        copy(i,dqn)
        
        if current_done
            
            break
            
        end
        
    end
    
    total_reward
    
end
        

#Run DQN algorithm

function main_DQN(dqn::DQN, env)
    
    e = 1

    scores = []

    while true

      reset!(env)

      total_reward = episode!(dqn,env)

      push!(scores, total_reward)

      print("Episode: $e | Score: $total_reward ")

      if e > 100

        last_100_mean = mean(scores[end-99:end])

        print("Last 100 episodes mean score: $last_100_mean")

        if last_100_mean > 195

          println("\nProblem solved!")

          break

        end
        
        if e > 200
                
          println("\nProblem unsolved!")
                
          break
        
        end

      end

      println()

      e += 1

    end
    
    e, scores
    
end
        
        
        
        
        
        

main_DQN (generic function with 1 method)